In [ ]:
import torch

def gaussian_weight_map(size, sigma):
    """Создает 2D карту весов с гауссовским распределением, центрированным в середине кропа."""
    y, x = torch.meshgrid(torch.arange(size), torch.arange(size), indexing='ij')
    center = (size - 1) / 2
    gauss_map = torch.exp(((x - center) * 2 + (y - center) * 2) / (2 * sigma ** 2))
    return gauss_map

def assemble_crops_with_gaussian_weights(crops, original_size, size, intersection_rate=0.2, sigma=0.4):
    # Создаем пустой тензор для результирующего изображения
    assembled_image = torch.zeros((crops.shape[1], original_size[1], original_size[2]), dtype=crops.dtype)
    
    h, w = original_size[1:3]
    step = int((1 - intersection_rate) * size)
    
    # Создаем карту весов один раз для всех кропов (можно менять sigma для настройки)
    weights = gaussian_weight_map(size, sigma * size).to(crops.device)  # Размер sigma адаптирован к размеру кропа
    
    count = torch.zeros((original_size[1], original_size[2]), dtype=torch.float32, device=crops.device)

    crop_index = 0
    for y1 in range(0, h, step):
        for x1 in range(0, w, step):
            if y1 + size > h:
                y1 = h - size
            if x1 + size > w:
                x1 = w - size
            
            # Извлекаем текущий кроп
            crop = crops[crop_index]
            crop_index += 1
            
            # Вставляем кроп в результирующее изображение с учетом весов
            assembled_image[..., y1:y1 + size, x1:x1 + size] += crop * weights
            count[y1:y1 + size, x1:x1 + size] += weights
            
    # Нормализуем результирующее изображение
    assembled_image /= count.unsqueeze(0)
    assembled_image[count == 0] = 0  # Обрабатываем пиксели, которые не были обновлены
    
    return assembled_image